# Match Meise Collectors to Wikidata Items Using *Cosine Similarity*

Basically we …
- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>

Technical Notes — Review Code perhaps:
- TODO refactor some data files to results….csv
- done implement: run matching on `canonical_string_fullname` vs. `canonical_string` (abbreviated) names
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", index_col=0, low_memory=False)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768.0,1826.0,NaN,NaN,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818.0,1904.0,NaN,NaN,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718.0,1790.0,NaN,NaN,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805.0,1860.0,NaN,NaN,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808.0,1880.0,NaN,NaN,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()

wd_matchtest
# cols = wd_matchtest.columns.tolist()

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


In [3]:
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

wd_matchtest_fullnames


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


## Load Collectors Data Set

**Data sources:**

- Jupyter Notebook for [create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [5]:
# unique names parsed already by ruby gem package: dwcagent

collectors = pd.read_csv("data/Meise_doi-10.15468-dl.ax9zkh/occurrence_recordedBy_eventDate_occurrenceIDs_20230830_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime or using pd.Periode(…)
print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(lambda x: pd.Period(x, freq='ms'))
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
13183,A,Buto,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000
14750,A,C.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000
16228,A,Carl G.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000
41708,A,H.M.L.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
57594,A,L. Do,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23993,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,6,http://www.botanicalcollections.be/specimen/BR...,1959-01-01 04:00:00.000,1958-08-13 00:00:00.000,1960-06-23 00:00:00.000
56392,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000
113216,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000
88015,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000


### Check Composition of Parsed Collector Data

In [6]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (17724 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
13183,A,Buto,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000
16228,A,Carl G.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000
88759,A,Scha,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
91698,A,Sti,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1852-01-01 00:00:00.000,1852-01-01 00:00:00.000,1852-01-01 00:00:00.000
61535,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91399,Čermak,Stan,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1926-05-01 00:00:00.000,1926-05-01 00:00:00.000,1926-05-01 00:00:00.000
91395,Čermák,Stan,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1925-10-12 00:00:00.000,1925-10-12 00:00:00.000,1925-10-12 00:00:00.000
93530,Şes,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1972-06-13 00:00:00.000,1972-06-13 00:00:00.000,1972-06-13 00:00:00.000
93531,Şesan-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,9,http://www.botanicalcollections.be/specimen/BR...,1972-04-06 08:00:00.000,1971-07-12 00:00:00.000,1973-06-12 00:00:00.000


In [7]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head())


----------------------------------------
show names with **particle** found 8071 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
21251,A,NaN,NaN,Da,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
104935,A Longhridge,G.,NaN,asper,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000
12160,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000
61535,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000
32703,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT



----------------------------------------
show names with **suffix** found 189 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
33442,Almeda,Frank,Jr.,NaN,NaN,NaN,NaN,NaN,2,http://www.botanicalcollections.be/specimen/BR...,1972-12-26 00:00:00.000,1972-12-20 00:00:00.000,1973-01-01 00:00:00.000
66249,Anderson,W.A.,Jr.,NaN,NaN,NaN,NaN,NaN,6,http://www.botanicalcollections.be/specimen/BR...,1930-08-09 16:00:00.000,1930-05-11 00:00:00.000,1930-09-24 00:00:00.000
53551,Atham,John,Jr,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1840-01-01 00:00:00.000,1840-01-01 00:00:00.000,1840-01-01 00:00:00.000
50514,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1886-07-23 00:00:00.000,1886-07-23 00:00:00.000,1886-07-23 00:00:00.000
58987,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN,3,http://www.botanicalcollections.be/specimen/BR...,1885-10-16 00:00:00.000,1884-05-21 00:00:00.000,1886-07-26 00:00:00.000



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max



----------------------------------------
show names with **appellation** found 1017 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
33175,A. Hardy,Lebrun,NaN,in,NaN,NaN,Fr.,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1869-01-01 00:00:00.000,1869-01-01 00:00:00.000,1869-01-01 00:00:00.000
112370,Abbon,G.,NaN,NaN,NaN,NaN,fr.,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1921-08-01 00:00:00.000,1921-08-01 00:00:00.000,1921-08-01 00:00:00.000
107943,Abione,S.,NaN,NaN,NaN,NaN,Mr,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1827-07-01 00:00:00.000,1827-07-01 00:00:00.000,1827-07-01 00:00:00.000
33126,Ahlfvengren,E.,NaN,NaN,NaN,NaN,Fr.,NaN,4,http://www.botanicalcollections.be/specimen/BR...,1892-06-27 18:00:00.000,1888-07-25 00:00:00.000,1896-05-25 00:00:00.000
112823,Alavoin,NaN,NaN,NaN,NaN,NaN,mr.,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1825-01-01 00:00:00.000,1825-01-01 00:00:00.000,1825-01-01 00:00:00.000


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [8]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)
collectors.tail()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
23993,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,"Žertová, A.",6,http://www.botanicalcollections.be/specimen/BR...,1959-01-01 04:00:00.000,1958-08-13 00:00:00.000,1960-06-23 00:00:00.000
56392,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000
113216,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000
88015,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000
113219,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,"Т. Королева. V. Petrovsky, В.Пертовский",1,http://www.botanicalcollections.be/specimen/BR...,1973-06-24 00:00:00.000,1973-06-24 00:00:00.000,1973-06-24 00:00:00.000


In [9]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,58,http://www.botanicalcollections.be/specimen/BR...,1897-12-11 03:54:04.445,1840-01-01 00:00:00.000,1995-10-26 00:00:00.000
1,A Longhridge,G.,NaN,asper,NaN,NaN,NaN,NaN,"A Longhridge, G.",1,http://www.botanicalcollections.be/specimen/BR...,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000
2,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000
3,A',NaN,NaN,NaN,NaN,NaN,NaN,NaN,A',1,http://www.botanicalcollections.be/specimen/BR...,1896-08-06 00:00:00.000,1896-08-06 00:00:00.000,1896-08-06 00:00:00.000
4,A'hlbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'hlbon,1,http://www.botanicalcollections.be/specimen/BR...,1864-05-01 00:00:00.000,1864-05-01 00:00:00.000,1864-05-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67110,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000
67111,Žertová,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Žertová, A.",20,http://www.botanicalcollections.be/specimen/BR...,1959-04-19 20:00:00.000,1958-07-09 00:00:00.000,1963-06-27 00:00:00.000
67112,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000
67113,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000


In [10]:
# TODO continue 2023-08-21 10:28:54
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [11]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [12]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['A L', ' Lo', 'Lon', 'ong', 'ngh', 'ghr', 'hri', 'rid', 'idg', 'dge', 'ge ', 'e G']
- data from match-test: ['Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' O ', 'O H']
- data from match-test (full name): ['188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr']


In [13]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))


(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


In [14]:
def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

In [15]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
matches

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 4.844703912734985 s
Cossim matches calculated after 6.340695381164551 s
Get all matches together ...
Done. Matches calculated after 6.752792596817017 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,10194,"Exell, A.W.","Exell, A.W.",1.0
1,5123,"Bäumler, J.A.","Bäumler, J.A.",1.0
2,16075,"Hoppe, D.H.","Hoppe, D.H.",1.0
3,5133,"Béguinot, A.","Béguinot, A.",1.0
4,26672,"Parmelee, J.A.","Parmelee, J.A.",1.0
...,...,...,...,...
38637,19331,"Kungu, E.M.","Chungu, D.",0.5
38638,24625,"Mortier, G. Const","Mortier, B.D.",0.5
38639,22358,Malwickz,"Chalwin, H.J.",0.5
38640,28607,"Rafael Torres, C.","Torres, C.M.",0.5


In [16]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

matches_fullnames

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 4.063576698303223 s
Cossim matches calculated after 5.320708990097046 s
Get all matches together ...
Done. Matches calculated after 5.410589933395386 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,1595,"Deyl, Miloš","Deyl, Miloš",1.0
1,986,"Carreiro, Bruno Tavares","Carreiro, Bruno Tavares",1.0
2,4295,"Löjtnant, Bernt","Løjtnant, Bernt",1.0
3,1759,"Eberhardt, Ursula","Eberhardt, Ursula",1.0
4,5545,"Razakamalala, Richardson","Razakamalala, Richardson",1.0
...,...,...,...,...
7040,6221,"Stengel, Stephanno","Gale, Stephan W",0.5
7041,940,"Calster, Eric","Walter, Erich",0.5
7042,4437,"Marten, GMv","Martens, Martin",0.5
7043,185,"Amable, Frè","Denesle, Jacques Amable Nicolas",0.5


### Create Output Results

Combine the matches data frame back to the (Meise) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [17]:
# # join (only) abbreviated name matches with collector source data
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True)
collectors_matches # 17552 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
2,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,0.0,"A Robyns, Br","Robyns, A.G.M.W.A.",0.572
6,A,C.,NaN,NaN,NaN,NaN,NaN,NaN,"A, C.",1,http://www.botanicalcollections.be/specimen/BR...,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,1.0,"A, C.","), A.C.(.",1.000
8,A,H.M.L.,NaN,NaN,NaN,NaN,NaN,NaN,"A, H.M.L.",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,2.0,"A, H.M.L.","Ros, A.H.M.",0.571
9,A,L. Do,NaN,NaN,NaN,NaN,NaN,NaN,"A, L. Do",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,3.0,"A, L. Do","Ros, A.L.d.",0.525
10,A,M.,NaN,NaN,NaN,NaN,NaN,NaN,"A, M.",1,http://www.botanicalcollections.be/specimen/BR...,1880-06-09 00:00:00.000,1880-06-09 00:00:00.000,1880-06-09 00:00:00.000,4.0,"A, M.","M, A.M.",0.658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67106,Żurowska,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Żurowska, M.",2,http://www.botanicalcollections.be/specimen/BR...,1952-08-07 12:00:00.000,1952-08-04 00:00:00.000,1952-08-11 00:00:00.000,38637.0,"Żurowska, M.","Borowska, A.",0.740
67107,Żytkowicz,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Żytkowicz, M.",6,http://www.botanicalcollections.be/specimen/BR...,2007-05-19 00:00:00.000,2007-05-19 00:00:00.000,2007-05-19 00:00:00.000,38638.0,"Żytkowicz, M.","Browicz, K.",0.570
67109,Željazova,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Željazova, B.",2,http://www.botanicalcollections.be/specimen/BR...,1956-05-18 00:00:00.000,1955-05-28 00:00:00.000,1957-05-09 00:00:00.000,38639.0,"Željazova, B.","Sijazov, M.M.",0.504
67113,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,38640.0,"Ǿllgaard, B.","Øllgaard, B.",1.000


In [18]:
# join (only) full name matches with collector source data
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname # 429 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,A,Carl G.,NaN,NaN,NaN,NaN,NaN,NaN,"A, Carl G.",1,http://www.botanicalcollections.be/specimen/BR...,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,0,"A, Carl G.","Fenner, Carl G",0.537
1,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,"A. Chevalier, Fleury",2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,1,"A. Chevalier, Fleury","Chevalier, Charles",0.534
2,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Dandois",1,http://www.botanicalcollections.be/specimen/BR...,1866-06-01 00:00:00.000,1866-06-01 00:00:00.000,1866-06-01 00:00:00.000,2,"A. Cogniaux, Dandois","Cogniaux, Alfred",0.600
3,A. Cogniaux,Gilbert,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Gilbert",2,http://www.botanicalcollections.be/specimen/BR...,1864-08-06 00:00:00.000,1864-08-06 00:00:00.000,1864-08-06 00:00:00.000,3,"A. Cogniaux, Gilbert","Cogniaux, Alfred",0.618
4,A. Cogniaux,Guilmot,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Guilmot",1,http://www.botanicalcollections.be/specimen/BR...,1867-05-19 00:00:00.000,1867-05-19 00:00:00.000,1867-05-19 00:00:00.000,4,"A. Cogniaux, Guilmot","Cogniaux, Alfred",0.585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7040,Øllgaard,Hans,NaN,NaN,NaN,NaN,NaN,NaN,"Øllgaard, Hans",1,http://www.botanicalcollections.be/specimen/BR...,1974-06-25 00:00:00.000,1974-06-25 00:00:00.000,1974-06-25 00:00:00.000,7040,"Øllgaard, Hans","Øllgaard, Hans",1.000
7041,Østrand,Eva,NaN,NaN,NaN,NaN,NaN,NaN,"Østrand, Eva",1,http://www.botanicalcollections.be/specimen/BR...,1939-07-20 00:00:00.000,1939-07-20 00:00:00.000,1939-07-20 00:00:00.000,7041,"Østrand, Eva","Strand, Elling Johan",0.532
7042,ällström,Joh,NaN,NaN,NaN,NaN,NaN,NaN,"ällström, Joh",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,7042,"ällström, Joh","Wahlström, Johan Erik",0.646
7043,ärther,Fürsten,NaN,NaN,NaN,NaN,NaN,NaN,"ärther, Fürsten",1,http://www.botanicalcollections.be/specimen/BR...,1848-09-10 00:00:00.000,1848-09-10 00:00:00.000,1848-09-10 00:00:00.000,7043,"ärther, Fürsten","Förster, Friedrich",0.532


In [19]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
6,A,C.,NaN,NaN,NaN,NaN,NaN,NaN,"A, C.",1,http://www.botanicalcollections.be/specimen/BR...,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,1.0,"A, C.","), A.C.(.",1.0
217,Aalto,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalto, M.",16,http://www.botanicalcollections.be/specimen/BR...,1976-07-05 11:12:00.000,1975-07-07 00:00:00.000,1976-09-09 00:00:00.000,77.0,"Aalto, M.","Aalto, M.",1.0
224,Aas,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Aas, O.",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,81.0,"Aas, O.","Aas, O.",1.0
248,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, A.",44,http://www.botanicalcollections.be/specimen/BR...,1934-10-15 01:39:18.621,1869-05-04 00:00:00.000,2001-08-10 00:00:00.000,91.0,"Abbas, A.","Abbas, A.",1.0
254,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",1,http://www.botanicalcollections.be/specimen/BR...,1932-09-01 00:00:00.000,1932-09-01 00:00:00.000,1932-09-01 00:00:00.000,95.0,"Abbe, E.C.","Abbe, E.C.",1.0


Save the plain name matching results only ...

In [20]:
if not os.path.exists('data'):
    print("Make data directory for saving …")
    os.makedirs('data')

# Set some global varialbes
# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20230913

this_output_file='data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
    this_timestamp_for_data
)

collectors_all_matches.to_csv(this_output_file)

print("Wrote plain name matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote plain name matches of collector names into data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_20230913.csv (8905 kB)


In [21]:
# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [22]:
# merge now with WikiData: the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [23]:
print("Show some name match examples (e.g. «Louis…» matching various names) …")
for testname in ['Louis', 'Abbot']:
    criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith(testname))    
    this_table=collectors_matches_g1_merged_wikidata[criterion].get([
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
        # 'canonical_string_fullname', 
        'itemLabel', 'wikidata_link',
        'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod', 'wyb', 'wye'
    ]).sort_values(by=['namematch_similarity'], ascending=[False])
    print("# ---------------------------------------------\n# «%s…» as test name, %d collector names begin with:" % (testname, criterion.sum()))    
    display(this_table)

Show some name match examples (e.g. «Louis…» matching various names) …
# ---------------------------------------------
# «Louis…» as test name, 72 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
26985,9,http://www.botanicalcollections.be/specimen/BR...,Louis-Marie,Louis-Marie,1.000,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1934-08-03 00:00:00.000,1952-07-01 00:00:00.000,1896.0,1978.0,NaN,NaN
19508,1,http://www.botanicalcollections.be/specimen/BR...,"Louis, A.","Louis, A.",1.000,A. Louis,http://www.wikidata.org/wiki/Q33682458,1944-03-10 00:00:00.000,1944-03-10 00:00:00.000,NaN,NaN,NaN,NaN
26994,2348,http://www.botanicalcollections.be/specimen/BR...,"Louis, A.M.","Louis, A.M.",1.000,Adriaan M. Louis,http://www.wikidata.org/wiki/Q21338327,1902-11-28 00:00:00.000,1995-05-04 00:00:00.000,1944.0,NaN,NaN,NaN
26987,22,http://www.botanicalcollections.be/specimen/BR...,"Louis-Marie, P.",Louis-Marie,0.914,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1921-08-19 00:00:00.000,1935-08-07 00:00:00.000,1896.0,1978.0,NaN,NaN
19518,1,http://www.botanicalcollections.be/specimen/BR...,Louis-Ar,"Louis, A.",0.879,A. Louis,http://www.wikidata.org/wiki/Q33682458,1948-06-23 00:00:00.000,1948-06-23 00:00:00.000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26973,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, H. Leters",Louis-Marie,0.508,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1905-06-27 00:00:00.000,1905-06-27 00:00:00.000,1896.0,1978.0,NaN,NaN
45177,2,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Jean Lebrun","Lebrun, Jean-Pierre",0.508,Jean-Pierre Lebrun,http://www.wikidata.org/wiki/Q21518496,1928-06-30 00:00:00.000,1928-07-22 00:00:00.000,1932.0,NaN,NaN,NaN
45572,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Arm",Louis-Marie,0.505,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1903-08-11 00:00:00.000,1903-08-11 00:00:00.000,1896.0,1978.0,NaN,NaN
42952,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Verhulst","Verhulst, Antoine",0.502,Antoine Verhulst,http://www.wikidata.org/wiki/Q113675938,1909-05-12 00:00:00.000,1909-05-12 00:00:00.000,1856.0,1931.0,NaN,NaN


# ---------------------------------------------
# «Abbot…» as test name, 6 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
442,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, A.T.D.","Abbott, A.T.D.",1.000,A. T. D. Abbott,http://www.wikidata.org/wiki/Q117328147,2010-01-07 00:00:00.000,2010-01-07 00:00:00.000,1936.0,2013.0,NaN,NaN
447,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, W.L.","Abbott, W.L.",1.000,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26 00:00:00.000,1922-02-26 00:00:00.000,1860.0,1936.0,NaN,NaN
444,2,http://www.botanicalcollections.be/specimen/BR...,"Abbott, R.","Abbott, R.J.",0.910,R.J. Abbott,http://www.wikidata.org/wiki/Q33660683,1995-01-27 00:00:00.000,1995-06-20 00:00:00.000,NaN,NaN,NaN,NaN
441,2,http://www.botanicalcollections.be/specimen/BR...,Abbott,"Abbott, G.",0.867,George Abbott,http://www.wikidata.org/wiki/Q47112598,1982-11-16 00:00:00.000,1982-11-16 00:00:00.000,NaN,NaN,NaN,NaN
446,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, W.C.","Abbott, W.L.",0.862,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,NaT,NaT,1860.0,1936.0,NaN,NaN
443,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, J. Richard","Abbott, J.R.",0.766,J. Richard Abbott,http://www.wikidata.org/wiki/Q18982386,1995-03-22 00:00:00.000,1995-03-22 00:00:00.000,1968.0,NaN,NaN,NaN


In [24]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'collectors_eventDate_mean', 'collectors_eventDate_min',
       'collectors_eventDate_max', 'old_index', 'namematch_source_data',
       'namematch_resource_data', 'namematch_similarity', 'item', 'itemLabel',
       'surname', 'initials', 'canonical_string', 'canonical_string_fullname',
       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob',
       'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link',
       'ipni_link', 'bionomia_link'],
      dtype='object')


In [25]:
collectors_matches_g1_merged_wikidata.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",1,...,Q5675465,1935.0,2003.0,NaN,NaN,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
1,Robyns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Robyns,25,...,Q5675465,1935.0,2003.0,NaN,NaN,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
2,Robyns,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Robyns, A.",1225,...,Q5675465,1935.0,2003.0,NaN,NaN,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
3,Robyns,A.G.,NaN,NaN,NaN,NaN,NaN,NaN,"Robyns, A.G.",4,...,Q5675465,1935.0,2003.0,NaN,NaN,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465
4,Robyns,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Robyns, D.",2,...,Q5675465,1935.0,2003.0,NaN,NaN,http://www.wikidata.org/wiki/Q5675465,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/8465-1,https://bionomia.net/Q5675465


In [26]:
# Select useful columns for data results
collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)

collectors_wikidata_cossim # comparison-match of «Kotschy, Karl Georg Th» (collector data) →← «Kotschy, T» (Wikidata) has only 0.5 similarity but corresponds to the correct person name we need

/tmp/ipykernel_34700/139787888.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)


,canonical_string_collector_parsed,family,given,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,item,canonical_string,...,harv,ipni,abbr,bionomia_id,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb
8,"A, C.",A,C.,1,http://www.botanicalcollections.be/specimen/BR...,"A, C.","), A.C.(.",1.0,http://www.wikidata.org/entity/Q118902945,"), A.C.(.",...,NaN,NaN,NaN,NaN,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,NaN,NaN,NaN
398,"Aalto, M.",Aalto,M.,16,http://www.botanicalcollections.be/specimen/BR...,"Aalto, M.","Aalto, M.",1.0,http://www.wikidata.org/entity/Q964994,"Aalto, M.",...,6655,30583-1,Aalto,NaN,1976-07-05 11:12:00.000,1975-07-07 00:00:00.000,1976-09-09 00:00:00.000,1939.0,NaN,NaN
409,"Aas, O.",Aas,O.,1,http://www.botanicalcollections.be/specimen/BR...,"Aas, O.","Aas, O.",1.0,http://www.wikidata.org/entity/Q36646127,"Aas, O.",...,74987,30584-1,Aas,NaN,NaT,NaT,NaT,NaN,NaN,NaN
420,"Abbas, A.",Abbas,A.,44,http://www.botanicalcollections.be/specimen/BR...,"Abbas, A.","Abbas, A.",1.0,http://www.wikidata.org/entity/Q60141229,"Abbas, A.",...,NaN,20034668-1,Al.Abbas,NaN,1934-10-15 01:39:18.621,1869-05-04 00:00:00.000,2001-08-10 00:00:00.000,NaN,NaN,NaN
421,"Abbas, A.",Abbas,A.,44,http://www.botanicalcollections.be/specimen/BR...,"Abbas, A.","Abbas, A.",1.0,http://www.wikidata.org/entity/Q88804360,"Abbas, A.",...,NaN,20034420-1,A.Abbas,NaN,1934-10-15 01:39:18.621,1869-05-04 00:00:00.000,2001-08-10 00:00:00.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39598,"Walderen, W.",Walderen,W.,1,http://www.botanicalcollections.be/specimen/BR...,"Walderen, W.","Walden, G.K.",0.5,http://www.wikidata.org/entity/Q15975611,"Walden, G.K.",...,100412,20017731-1,Walden,NaN,1884-06-12 00:00:00.000,1884-06-12 00:00:00.000,1884-06-12 00:00:00.000,NaN,NaN,NaN
39867,"Wehrkamp, D.",Wehrkamp,D.,1,http://www.botanicalcollections.be/specimen/BR...,"Wehrkamp, D.","Oskamp, D.L.",0.5,http://www.wikidata.org/entity/Q21522455,"Oskamp, D.L.",...,63303,7183-1,Oskamp,NaN,NaT,NaT,NaT,1768.0,NaN,NaN
40404,"Wilwerth, E.",Wilwerth,E.,67,http://www.botanicalcollections.be/specimen/BR...,"Wilwerth, E.","Werth, E.",0.5,http://www.wikidata.org/entity/Q1336985,"Werth, E.",...,32071,11568-1,Werth,Q1336985,1901-01-26 02:28:57.932,1896-01-01 00:00:00.000,1912-06-01 00:00:00.000,1869.0,1958.0,NaN
46843,"Winter, Pharm",Winter,Pharm,4,http://www.botanicalcollections.be/specimen/BR...,"Winter, Pharm","Winter, Pieter J D",0.5,http://www.wikidata.org/entity/Q19003385,"Winter, P.J.D.",...,89410,37283-1,P.J.D.Winter,NaN,1861-08-01 00:00:00.000,1861-08-01 00:00:00.000,1861-08-01 00:00:00.000,1964.0,NaN,NaN


In [27]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# meise_collectors_cosine-similarity_wikidata-botanists_%s.csv
this_output_file='data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
    this_timestamp_for_data
)

collectors_wikidata_cossim.to_csv(this_output_file)

print("Wrote matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names into data/results_meise_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_20230913.csv (14291 kB)


TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, see https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))